#### Submitted by Sanghak Jeon
#### Submitted on 02/19/2024
<br>

#### Contacts
Email         : sanghakj@princeton.edu
Mobile        : 609-558-6291 
Linkedin      : <a href = https://www.linkedin.com/in/sanghak-jeon/>https://www.linkedin.com/in/sanghak-jeon/</a>

## Contents

* Dataset analysis
* Logistics
    - n-gram
    - minor details
* Trials and Ideas
* Result

# 1. Dataset analysis

The file "words_250000_train.txt" was given, so I tried to check possible glitches/data corruptions. There was none.

* There are exactly 227300 words.
* The maximum length of the word from "words_250000_train.txt" is 29.
* The distribution by word's length is
    3: 2201, 6: 19541, 4: 5287, 5: 11274, 8: 30452,
    7: 25948, 10: 26953, 9: 30906, 11: 22786, 12: 18178,
    13: 12956, 15: 5211, 14: 8710, 20: 225, 17: 1775,
    16: 3143, 2: 264, 21: 98, 18: 859, 19: 441,
    25: 3, 22: 44, 1: 17, 23: 14, 29: 2,
    24: 9, 28: 1, 27: 2
    (these are unsorted, because I calculated with a dictionary structure.)

In [1]:
import time, collections

alphabets ={'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'}
vowels = {'a', 'e', 'i', 'o', 'u'}
semivowels = {'y'}                          # linguistically y sometimes behave as a vowel
consonants = list(set(alphabets) - set(vowels) - set(semivowels))

def build_dictionary(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

# dictionary loading
full_dictionary_location = "words_250000_train.txt"
full_dictionary = build_dictionary(full_dictionary_location)

# dictionary length counting
length = collections.defaultdict(int)
for word in full_dictionary:
    length[len(word)]+=1
print(length)

defaultdict(<class 'int'>, {3: 2201, 6: 19541, 4: 5287, 5: 11274, 8: 30452, 7: 25948, 10: 26953, 9: 30906, 11: 22786, 12: 18178, 13: 12956, 15: 5211, 14: 8710, 20: 225, 17: 1775, 16: 3143, 2: 264, 21: 98, 18: 859, 19: 441, 25: 3, 22: 44, 1: 17, 23: 14, 29: 2, 24: 9, 28: 1, 27: 2})


Furthermore, I need some distributional information on consonants for later.

In [1]:
# 4-grams with all consonants
totalcount, count, wordcount, iscccc = 0, 0, 0, False
for word in full_dictionary:
    iscccc = False
    if len(word) >= 4:
        for i in range(len(word)-3):
            totalcount += 1
            if (word[i] in consonants) and (word[i+1] in consonants) and (word[i+2] in consonants) and (word[i+3] in consonants):
                # print(f' sequence       "{word[i:i+4]}", from {word}')
                count += 1
                iscccc = True
    wordcount += iscccc
print(f' total {totalcount} number of 4-grams and total {count} number of 4-consonant-only-grams. total {wordcount} '
      f'number of words including those.')


# 5-grams with all consonants
totalcount, count, wordcount, isccccc = 0, 0, 0, False
for word in full_dictionary:
    isccccc = False
    if len(word) >= 5:
        for i in range(len(word)-4):
            totalcount += 1
            if ((word[i] in consonants) and (word[i+1] in consonants) and (word[i+2] in consonants) and (word[i+3] in consonants) and (word[i+4] in consonants)):
                # print(f' sequence       "{word[i:i+5]}", from {word}')
                count += 1
                isccccc = True
    wordcount += isccccc
print(f' total {totalcount} number of 5-grams and total {count} number of 5-consonant-only-grams. total {wordcount} '
      f'number of words including those.')


# 6-grams with all consonants
totalcount, count, wordcount, iscccccc = 0, 0, 0, False
for word in full_dictionary:
    iscccccc = False
    if len(word) >= 6:
        for i in range(len(word)-5):
            totalcount += 1
            if ((word[i] in consonants) and (word[i+1] in consonants) and (word[i+2] in consonants) and (word[i+3] in consonants) and (word[i+4] in consonants) and (word[i+5] in consonants)):
                # print(f' sequence       "{word[i:i+6]}", from {word}')
                count += 1
                iscccccc = True
    wordcount += iscccccc
print(f' total {totalcount} number of 6-grams and total {count} number of 6-consonant-only-grams. total {wordcount} '
      f'number of words including those.')

NameError: name 'full_dictionary' is not defined

# 2. Logistics

#### 2.1. n-gram

The most common approach (and also naive) is the n-gram (<a href = https://en.wikipedia.org/wiki/N-gram>https://en.wikipedia.org/wiki/N-gram</a>). In a real hangman 
game, often people choose 'e' for their first guess, not only 'e' is a vowel but also statistically 'e' comes out 
more frequently. n-gram idea is a generalization of frequency-based likelihood calculation. As an example of 
2-gram, we can calculate all frequencies over possible two consecutive letters, from "aa" to "zz". We can think a 
chunk of two letters such as "ch" or "ll" would be more frequent compare to "qv" or "iw". This idea can be 
generalized into more letters: 3-grams would detect phrases such as "-ing", "pre-", and "-ght-". 4-grams would detect
phrases such as "-tion", "-ness", and "auto-". 5-grams would detect "under-", "-ative", "-phone".
<br>

Adapting n-gram algorithm brings two natural questions:
* How many different n-grams do we need? Will 10-grams be helpful?
* How are we going to 'interpolate/mix' results from different n-grams? How much should we consider more about 
2-gram results compare to 1-gram results?
<br>

Theoretically, there are 26^10 possible alphabet 10-grams. In order to count the frequency of 10-grams, the word must
 consist of more than 10 letters, and those collected 10-grams will be evenly distributed unless there is a common 
 prefix/suffix consists of 10 or more letters. Hence the usefulness of n-grams must rely on how many common 
 prefixes/suffixes are there with n or more letters. 
<br>

I have checked these websites:
* <a href = https://en.wikipedia.org/wiki/English_prefix>https://en.wikipedia.org/wiki/English_prefix</a>
* <a href = https://dictionary.cambridge.org/us/grammar/british-grammar/prefixes>https://dictionary.cambridge
.org/us/grammar/british-grammar/prefixes</a>

I could investigate this further, but I decided to stop because this might violate external-material training rule. 
Also empirically I can imagine 5 letter prefix/suffix like "extra-", "hyper-", "super-", "trans-", "-ology", 
"-ation", "-pathy" while I cannot think of 6+ letter prefix/suffix other than "contra-". Hence I decided to only 
consider 1-gram to 5-gram.
<br>

For the second question, let's consider "webs_te". If there are enough "-site-"s on the original dictionary, 
guessing according to 4-gram would give a better prediction compare to guessing only by the frequency of each letters
(1-gram). Hence we can think the weights for n-gram must be larger as n grows. Furthermore, as n-gram information 
becomes sparser as n grows(since there are $26^n$ possible configurations), the weights should be even larger. We can 
try to give this weight information as a function of (disclosed letters)/(total letters), (for "webs_te" it becomes 
6/7), but I decided not to. The weights on n-gram should grow exponentially on n (compare to $26^n$), so I decided to 
consider $[1, 2, 4, 8, 16]$ as a baseline.

#### 2.2. code details 

Below is the code that I used for the final submission. Other than '__init__', 'guess', 'frequency_calc', 
'fivegram_adder' to 'unigram_adder', I intentionally substituted all contents to "pass". 'build_dictionary' method was 
premade and I didn't touch it, but I left it untouched because 'guess' calls it.

In [4]:
### hangman API

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        
        # training data importing        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        # information for guesses
        self.current_dictionary = []
        self.guessed_letters = []
        self.alphabets = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
                          's', 't', 'u', 'v', 'w', 'x', 'y', 'z'}
        
    @staticmethod
    def determine_hangman_url():
        pass

    def guess(self, word):
        # word input example: "a p p l e " == " _ p p _ e"
        # should return a letter
        
        clean_word = word[::2].replace("_",".")     # clean_word: ".pp.e"
        
        self.current_trial_count = len(self.guessed_letters)
        self.wrong_guesses = list(set(self.guessed_letters) - set(clean_word))
        self.correct_guesses = list(set(self.guessed_letters) - set(self.wrong_guesses))
            # beware: from the example 'p' does not count twice
        empty_sites = 0
        for letter in clean_word:
            empty_sites += (letter == ".")
        self.empty_sites = empty_sites
                
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        
        if (len(self.guessed_letters) > 0 
                and self.guessed_letters[-1] in self.wrong_guesses 
                and len(self.wrong_guesses) >= 3):
            new_dictionary = []
            for word in current_dictionary:
                if not set(word).intersection(self.wrong_guesses):
                    new_dictionary += [word]
                  
            # overwrite old possible words dictionary with updated version
            self.current_dictionary = new_dictionary
        
        (self.unigram_frequency, self.twogram_frequency, self.threegram_frequency, self.fourgram_frequency, self
        .fivegram_frequency) = (self.frequency_calc(self.current_dictionary))
        self.weight = [1, 2, 4, 8, 16]
            # sum should be 1; weight[0], weight[1], weight[2] corresponds to unigram/2-gram/3-gram
        
        self.final_prob = [[0]*26 for _ in range(self.empty_sites)]
        self.fivegram_adder(clean_word)
        self.fourgram_adder(clean_word)
        self.threegram_adder(clean_word)
        self.twogram_adder(clean_word)
        self.unigram_adder(clean_word)
        self.final_prob = np.array(self.final_prob)
        self.final_prob = self.final_prob / self.final_prob.sum(axis = 1, keepdims=True)
        self.final_prob = self.final_prob.sum(axis = 0)
        
        guess_letter = '!'

        temp, letter = 0, 0
        for letter in self.alphabets:
            if temp < self.final_prob[ord(letter) - ord('a')] and letter not in self.guessed_letters:
                temp = self.final_prob[ord(letter) - ord('a')]
                guess_letter = letter
        return guess_letter
        
    def frequency_calc(self, dictionary):
        # returns unigram / twogram / threegram / fourgram / fivegram frequencies
        
        unigram   = collections.defaultdict(lambda: collections.defaultdict(lambda : collections.defaultdict(int)))
        twogram   = collections.defaultdict(lambda: collections.defaultdict(int))
        threegram = collections.defaultdict(lambda: collections.defaultdict(lambda : collections.defaultdict(int)))
        fourgram  = collections.defaultdict(lambda: collections.defaultdict(lambda : collections.defaultdict(lambda : collections.defaultdict(int))))
        fivegram  = collections.defaultdict(lambda: collections.defaultdict(lambda : collections.defaultdict(lambda : collections.defaultdict(lambda : collections.defaultdict(int)))))
            # unigram[wordlength][letterlocation][letter] = ...
            # twogram[letter_1][letter_2] = ...
            # threegram[letter_1][letter_2][letter_3] = ...
            # fourgram[letter_1][letter_2][letter_3][letter_4] = ...
            # fivegram[letter_1][letter_2][letter_3][letter_4][letter_5] = ...
        
        for word in dictionary:
            current_length = len(word)
            if current_length == 1:
                unigram[current_length][0][word[0]] += 1
            elif current_length == 2:
                unigram[current_length][0][word[0]] += 1
                unigram[current_length][1][word[1]] += 1
                twogram[word[0]][word[1]] += 1
            elif current_length == 3:
                unigram[current_length][0][word[0]] += 1
                unigram[current_length][1][word[1]] += 1
                unigram[current_length][2][word[2]] += 1
                twogram[word[0]][word[1]] += 1
                twogram[word[1]][word[2]] += 1
                threegram[word[0]][word[1]][word[2]] += 1
            elif current_length == 4:
                unigram[current_length][0][word[0]] += 1
                unigram[current_length][1][word[1]] += 1
                unigram[current_length][2][word[2]] += 1
                unigram[current_length][3][word[3]] += 1
                twogram[word[0]][word[1]] += 1
                twogram[word[1]][word[2]] += 1
                twogram[word[2]][word[3]] += 1
                threegram[word[0]][word[1]][word[2]] += 1
                threegram[word[1]][word[2]][word[3]] += 1
                fourgram[word[0]][word[1]][word[2]][word[3]] += 1
            
            else : # current_length >= 5
                for i in range(current_length - 4):
                    unigram[current_length][i][word[i]] += 1
                    twogram[word[i]][word[i+1]] += 1
                    threegram[word[i]][word[i+1]][word[i+2]] += 1
                    fourgram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1
                    fivegram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]] += 1
                # corner cases
                unigram[current_length][current_length-1][word[-1]] += 1
                unigram[current_length][current_length-2][word[-2]] += 1
                unigram[current_length][current_length-3][word[-3]] += 1
                unigram[current_length][current_length-4][word[-4]] += 1
                twogram[word[-2]][word[-1]] += 1
                twogram[word[-3]][word[-2]] += 1
                twogram[word[-4]][word[-3]] += 1
                threegram[word[-3]][word[-2]][word[-1]] += 1
                threegram[word[-4]][word[-3]][word[-2]] += 1
                fourgram[word[-4]][word[-3]][word[-2]][word[-1]] += 1
                
        return unigram, twogram, threegram, fourgram, fivegram

    def fivegram_adder(self, word):
        '''
        counts all possible fivegram_frequency and directly add to self.final_prob
        returns nothing
        '''
        wordshape, count = [0]*len(word), 0
        for i in range(len(word)):
            if word[i] == ".": 
                wordshape[i] = count
                count += 1
                
        fiveprob, count = [[0]*26 for _ in range(self.empty_sites)], [0]*self.empty_sites
        for i in range(len(word) - 4):
            if word[i] != "." and word[i+1] != "." and word[i+2] != "." and word[i+3] != "." and word[i+4] == ".":
                # ****. form
                for letter in self.alphabets:
                    fiveprob[wordshape[i+4]][ord(letter) - ord('a')] += self.fivegram_frequency[word[i]][word[i+1]][word[i+2]][word[i+3]][letter]
                    count[wordshape[i+4]] += self.fivegram_frequency[word[i]][word[i+1]][word[i+2]][word[i+3]][letter]
            elif word[i] != "." and word[i+1] != "." and word[i+2] != "." and word[i+3] == "." and word[i+4] != ".":
                # ***.* form
                for letter in self.alphabets:
                    fiveprob[wordshape[i+3]][ord(letter) - ord('a')] += self.fivegram_frequency[word[i]][word[i+1]][word[i+2]][letter][word[i+4]]
                    count[wordshape[i+3]] += self.fivegram_frequency[word[i]][word[i+1]][word[i+2]][letter][word[i+4]]
            elif word[i] != "." and word[i+1] != "." and word[i+2] == "." and word[i+3] != "." and word[i+4] != ".":
                # **.** form
                for letter in self.alphabets:
                    fiveprob[wordshape[i+2]][ord(letter) - ord('a')] += self.fivegram_frequency[word[i]][word[i+1]][letter][word[i+3]][word[i+4]]
                    count[wordshape[i+2]] += self.fivegram_frequency[word[i]][word[i+1]][letter][word[i+3]][word[i+4]]  
            elif word[i] != "." and word[i+1] == "." and word[i+2] != "." and word[i+3] != "." and word[i+4] != ".":
                # *.*** form
                for letter in self.alphabets:
                    fiveprob[wordshape[i+1]][ord(letter) - ord('a')] += self.fivegram_frequency[word[i]][letter][word[i+2]][word[i+3]][word[i+4]]
                    count[wordshape[i+1]] += self.fivegram_frequency[word[i]][letter][word[i+2]][word[i+3]][word[i+4]]
            elif word[i] == "." and word[i+1] != "." and word[i+2] != "." and word[i+3] != "." and word[i+4] != ".":
                # .**** form
                for letter in self.alphabets:
                    fiveprob[wordshape[i]][ord(letter) - ord('a')] += self.fivegram_frequency[letter][word[i+1]][word[i+2]][word[i+3]][word[i+4]]
                    count[wordshape[i]] += self.fivegram_frequency[letter][word[i+1]][word[i+2]][word[i+3]][word[i+4]]            
        
        # normalization
        for i in range(self.empty_sites):
            if count[i] != 0:
                for letter in self.alphabets:
                    self.final_prob[i][ord(letter) - ord('a')] += self.weight[4] * 1.0* fiveprob[i][ord(letter) - ord('a')]/count[i]

    def fourgram_adder(self, word):
        '''
        counts all possible fourgram_frequency and directly add to self.final_prob
        returns nothing
        '''
        wordshape, count = [0]*len(word), 0
        for i in range(len(word)):
            if word[i] == ".": 
                wordshape[i] = count
                count += 1
        
        fourprob, count = [[0]*26 for _ in range(self.empty_sites)], [0]*self.empty_sites
        for i in range(len(word) - 3):
            if word[i] != "." and word[i+1] != "." and word[i+2] != "." and word[i+3] == ".":       # ***. form
                for letter in self.alphabets:
                    fourprob[wordshape[i+3]][ord(letter) - ord('a')] += self.fourgram_frequency[word[i]][word[i+1]][word[i+2]][letter]
                    count[wordshape[i+3]] += self.fourgram_frequency[word[i]][word[i+1]][word[i+2]][letter]
            elif word[i] != "." and word[i+1] != "." and word[i+2] == "." and word[i+3] != ".":     # **.* form
                for letter in self.alphabets:
                    fourprob[wordshape[i+2]][ord(letter) - ord('a')] += self.fourgram_frequency[word[i]][word[i+1]][letter][word[i+3]]
                    count[wordshape[i+2]] += self.fourgram_frequency[word[i]][word[i+1]][letter][word[i+3]]
            elif word[i] != "." and word[i+1] == "." and word[i+2] != "." and word[i+3] != ".":     # *.** form
                for letter in self.alphabets:
                    fourprob[wordshape[i+1]][ord(letter) - ord('a')] += self.fourgram_frequency[word[i]][letter][word[i+2]][word[i+3]]
                    count[wordshape[i+1]] += self.fourgram_frequency[word[i]][letter][word[i+2]][word[i+3]]
            elif word[i] == "." and word[i+1] != "." and word[i+2] != "." and word[i+3] != ".":     # .*** form
                for letter in self.alphabets:
                    fourprob[wordshape[i]][ord(letter) - ord('a')] += self.fourgram_frequency[letter][word[i+1]][word[i+2]][word[i+3]]
                    count[wordshape[i]] += self.fourgram_frequency[letter][word[i+1]][word[i+2]][word[i+3]]
        
        # normalization
        for i in range(self.empty_sites):
            if count[i] != 0:
                for letter in self.alphabets:
                    self.final_prob[i][ord(letter) - ord('a')] += self.weight[3] * 1.0 * fourprob[i][ord(letter) - ord('a')]/count[i]

    def threegram_adder(self, word):
        '''
        counts all possible threegram_frequency and directly add to self.final_prob
        returns nothing
        '''
        wordshape, count = [0]*len(word), 0
        for i in range(len(word)):
            if word[i] == ".": 
                wordshape[i] = count
                count += 1
        
        threeprob, count = [[0]*26 for _ in range(self.empty_sites)], [0]*self.empty_sites
        for i in range(len(word) - 2):
            if word[i] != "." and word[i+1] != "."  and word[i+2] == ".":       # **. form
                for letter in self.alphabets:
                    threeprob[wordshape[i+2]][ord(letter) - ord('a')] += self.threegram_frequency[word[i]][word[i+1]][letter]
                    count[wordshape[i+2]] += self.threegram_frequency[word[i]][word[i+1]][letter]
            elif word[i] != "." and word[i+1] == "."  and word[i+2] != ".":     # *.* form
                for letter in self.alphabets:
                    threeprob[wordshape[i+1]][ord(letter) - ord('a')] += self.threegram_frequency[word[i]][letter][word[i+2]]
                    count[wordshape[i+1]] += self.threegram_frequency[word[i]][letter][word[i+2]]
            elif word[i] == "." and word[i+1] != "."  and word[i+2] != ".":     # .** form
                for letter in self.alphabets:
                    threeprob[wordshape[i]][ord(letter) - ord('a')] += self.threegram_frequency[letter][word[i+1]][word[i+2]]
                    count[wordshape[i]] += self.threegram_frequency[letter][word[i+1]][word[i+2]]
        
        # normalization
        for i in range(self.empty_sites):
            if count[i] != 0:
                for letter in self.alphabets:
                    self.final_prob[i][ord(letter) - ord('a')] += self.weight[2] * 1.0 * threeprob[i][ord(letter) - ord('a')]/count[i]
            
    def twogram_adder(self, word):
        '''
        counts all possible twogram_frequency and directly add to self.final_prob
        returns nothing
        '''
        wordshape, count = [0]*len(word), 0
        for i in range(len(word)):
            if word[i] == ".": 
                wordshape[i] = count
                count += 1
        
        twoprob, count = [[0]*26 for _ in range(self.empty_sites)], [0]*self.empty_sites
        for i in range(len(word) - 1):
            if word[i] != "." and word[i+1] == ".":       # *. form
                for letter in self.alphabets:
                    twoprob[wordshape[i+1]][ord(letter)-ord('a')] += self.twogram_frequency[word[i]][letter]
                    count[wordshape[i+1]] += self.twogram_frequency[word[i]][letter]
            elif word[i] == "." and word[i+1] != ".":     # .* form
                for letter in self.alphabets:
                    twoprob[wordshape[i]][ord(letter)-ord('a')] += self.twogram_frequency[letter][word[i+1]]
                    count[wordshape[i]] += self.twogram_frequency[letter][word[i+1]]
        
        # normalization
        for i in range(self.empty_sites):
            if count[i] != 0:
                for letter in self.alphabets:
                    self.final_prob[i][ord(letter) - ord('a')] += self.weight[1] * 1.0 * twoprob[i][ord(letter) - ord('a')]/count[i]
    
    def unigram_adder(self, word):
        '''
        counts all possible onegram_frequency and directly add to self.final_prob
        returns nothing
        '''
        wordshape, count = [0]*len(word), 0
        for i in range(len(word)):
            if word[i] == ".": 
                wordshape[i] = count
                count += 1
                
        oneprob, count = [[0]*26 for _ in range(self.empty_sites)], [0]*self.empty_sites
        for i in range(len(word)):
            if word[i] == ".":
                if i == 0 or i == len(word) - 1:        # empty_sites for first/last letters
                    for letter in self.alphabets:
                        oneprob[wordshape[i]][ord(letter) - ord('a')] += self.unigram_frequency[len(word)][i][letter]
                        count[wordshape[i]] += self.unigram_frequency[len(word)][i][letter]
                else:                                   # other empty_sites. apply the usual frequency information.
                    for letter in self.alphabets:
                        for j in range(len(word)):
                            oneprob[wordshape[i]][ord(letter) - ord('a')] += self.unigram_frequency[len(word)][j][letter]
                            count[wordshape[i]] += self.unigram_frequency[len(word)][j][letter]
        
        # normalization
        for i in range(self.empty_sites):
            if count[i] != 0:
                for letter in self.alphabets:
                    self.final_prob[i][ord(letter) - ord('a')] += self.weight[0] * 1.0 * oneprob[i][ord(letter) - ord('a')]/count[i]

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

# 3. Trials and Ideas

#### 3.1. (1-gram) + (2-gram) + (3-gram) + (4-gram) only

This is the very first model that I have tried. As I couldn't easily imagine 5 letter prefixes/suffixes, I tried only
 upto 4-grams, without 5-gram information. I have tried 100 examples on the server, and I got 45 out of 100. After I 
 tried models with 5-grams and realized there are meaningful 5-grams, I decided not to work on this model. All models
  with 5-gram I have tried exceeds 50% win rate, with more than 500 games.
 

#### 3.2. soft n-gram

3-gram does not work if there are two or more unknown letters. For example, if we want to guess a letter from "ab_ent", 
the 3-gram model calculates frequency over "aba", "abb", ..., "abz", "bae", "bbe", ..., "bze", "aen", "ben", ..., 
"zen" but the model cannot calculate frequencies from "a__ent". I thought about generalizing this n-gram idea and tried
 to calculate frequency over "aaa" ... "azz" (total $26^2$ ways) and "aae" ... "zze" (total $26^2$ ways). However 
 this didn't work well, and I think it is because the weight for 3-gram is larger than 1-gram, and random sequences like "aaa" or "azz" 
weaken the information from 1-gram. I decided not to apply this n-gram idea with multiple unknown characters.

#### 3.3. wordlength

My model guesses the first letter according to the frequency over the words with same length, not over all words from
 the dictionary. This is because the most frequent letter differs by the length of the given word(<a href = http://www.datagenetics.com/blog/april12012/>http://www.datagenetics.com/blog/april12012/</a>).
Hence I decided to also consider the length of the given word and keep restricted for unigram_frequency during the 
entire guess. We need to rigorously check with conditional probability argument to verify, but empirically this gives
 better prediction score.

Some people suggests to apply the word length information to 2-gram so that we can selectively use 2-grams too, but 
this turned out to lower the prediction score, so I decided not to.
<br>

Similarly we can compare with the model with unigram not using word length information, but this also lowers the 
prediction score.

#### 3.4. dictionary reselection

Suppose the model guess a letter "e" and it is wrong. This also includes information that it does not have the letter,
 and we can calibrate "self.current_dictionary" by excluding all words with "e". We might try to exclude words 
 every time we incorrectly guesses, but this decreases the program speed, and (surprisingly) the prediction score. I 
 think this happens because each wrong guess excludes so many words and the remaining frequency information becomes too 
 sparser. At some point we need to adjust the dictionary for better prediction score, but I decided not to perform the calibration 
 process at the early stages. The model only recalibrates when it's previous guess is wrong and there are only 1 or 2 
 future wrong guesses are remaining.

#### 3.5. Consonant sequence analysis

For simplicity, I would like to write C and V for consonants and vowels.
<br>

In a linguistic sense, consecutive consonants are hard to pronounce. This makes a word with consecutive consonants 
are uncommon. For example, Typical 4-letter words have a form of CVCV either CVVC, which leads "CC" configuration is 
relatively less common. Of course there are combined consonants such as "ch" and "sh", which are even a single letter
 on Phonetic Alphabets, and there is a possibility that the given word can be split into two words which finishes 
 with CC and starts with CC, such as "backspace"(CCCC), or even "worthwhile"(CCCCC).
<br>

Sometimes 'y' works as a vowel ("tiny", "flurry") so I exclude 'y' from consonants, and tried to calculate 
frequencies of CCCCC and CCCCCC. There are 540 words with CCCCC and 43 words with CCCCCC out of 227300 words. I 
modified the model less likely to choose a letter which generates CCCCC, and even exclude CCCCCC. However, this does 
not enhance model's prediction score(the score was not statistically significantly better than the best among the 
others) while it significantly slows down the program. Hence I decided not to implement this idea to the model. 

#### 3.6. padding at the beginning / ending

One of the common defect of n-gram model is that does not consider the location of each letter. Consider a word 
"_bsent", which is "absent". 2-gram will give you the ratio of (number of "_b"s)/(sum of the numbers of "*b"s), 
where * is a wildcard(can be any single letter). This calculation is not utilizing the fact that this blank is at 
the front. Heuristically we can guess 'a' with this fact, so it is possible to add some paddings on each
words and also calculate the frequency over those.
<br>
  
For example, for 2-grams, we can modify the word "absent" to "{{absent{{", and add "{a", "ab", "bs", "se", "en", 
"nt", "t{" to the frequency information. This becomes particularly helpful in case of "_bsent" -> "absent", by using 
two 2-grams, "{_" and "_b".
<br>

However, this turned out to give too biased results at the early stages because only the first letter and the last 
letter can utilize 2-gram information while all the other letters cannot. I decided to implement this first/last 
letter information in another way, and to this end I devised the below, 3.1.7.

#### 3.7. probability calculating each sites vs probability calculating only over alphabets

I thought the first/last letter frequency over all letters must be different from the usual letter frequency, 
so I decided to calculate those separately and replace usual 1-gram results to these. However this idea requires me 
to calculate probability of each alphabet on every site, and this requires one more step to think of because the 
model only guesses one letter each time, not specifying possible sites with the letter.
<br>

This takes longer calculation time, but I decided to consider (26, self.empty_sites) size of probability matrix, not 
just (26, 1). After I finish collecting information from 5-gram to 1-gram, the model normalizes probability on each 
empty site, and sum them up by each letter. The model chooses the letter with the largest probability sum from each 
empty site. 

Even though this takes longer time, the model showed better performance over practice set, around 58%.

# 4. Result

I submitted the final test at 2/20 afternoon. Total calculation took 4 hours and 38 minutes, and the result was 568 
correct word guesses out of 1000 trials.